### **Text Classification by Fine-Tuning Language Model**

### Output

**1. Data Loading**

In [ ]:
!pip install simpletransformers pandas scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00
   

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel
import torch

In [ ]:
data = pd.read_excel('/content/dataset_NLP.xlsx')

print(data.head())
print(data.info())
print(data['Prediction'].value_counts())

                                               Input Prediction
0                                                NaN        NaN
1  Tesla's stock experienced a remarkable 5% surg...   Positive
2  Concerns over a looming market crash have inte...   Negative
3  The S&P 500 index closed the day without any s...    Neutral
4  Apple reported that its latest iPhone model ac...   Positive
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007 entries, 0 to 1006
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Input       1006 non-null   object
 1   Prediction  1006 non-null   object
dtypes: object(2)
memory usage: 15.9+ KB
None
Prediction
Positive    357
Negative    341
Neutral     308
Name: count, dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

data = pd.read_excel('/content/dataset_NLP.xlsx')

data = data.dropna(subset=['Input', 'Prediction'])

data = data.rename(columns={"Input": "text", "Prediction": "label"})

label_mapping = {"Positive": 1, "Negative": 0, "Neutral": 2}
data = data[data['label'].isin(label_mapping)]

data["label"] = data["label"].map(label_mapping)

train_data, val_data = train_test_split(data, test_size=0.2, random_state=42, stratify=data["label"])

train_df = train_data[['text', 'label']].copy()
val_df = val_data[['text', 'label']].copy()

print(f"Training size: {len(train_df)}, Validation size: {len(val_df)}")

Training size: 804, Validation size: 202


**2. Text Processing**

In [ ]:
import re

def clean_text(text):
    text = text.lower()


    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.strip()
    return text

train_df['text'] = train_df['text'].apply(clean_text)
val_df['text'] = val_df['text'].apply(clean_text)

print(train_df.head())

                                                  text  label
929  crude oil price volatility continues to impact...      0
317  fears of an economic downturn and sluggish gdp...      0
538  reliances strategic entry into the semiconduct...      1
742  reliance retails aggressive expansion into the...      1
365  foreign institutional investors fiis poured su...      1


**3. Text Embedding using BERT and RoBERTa**

In [ ]:
bert_model = ClassificationModel('bert', 'bert-base-uncased', num_labels=3, use_cuda=False)
roberta_model = ClassificationModel('roberta', 'roberta-base', num_labels=3, use_cuda=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

**4. Model Training with BERT and RoBERTa**

In [ ]:
import torch

use_cuda = torch.cuda.is_available()

from simpletransformers.classification import ClassificationArgs

model_args = ClassificationArgs(
    num_train_epochs=3,
    train_batch_size=8,
    eval_batch_size=8,
    learning_rate=3e-5,
    max_seq_length=128,
    weight_decay=0.01,
    warmup_steps=0,
    logging_steps=50,
    save_steps=200,
    overwrite_output_dir=True
)

bert_model = ClassificationModel('bert', 'bert-base-uncased', num_labels=3, args=model_args, use_cuda=use_cuda)
bert_model.train_model(train_df)

roberta_model = ClassificationModel('roberta', 'roberta-base', num_labels=3, args=model_args, use_cuda=use_cuda)
roberta_model.train_model(train_df)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:882: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Running Epoch 1 of 3:   0%|          | 0/101 [00:00<?, ?it/s]

...

Running Epoch 1 of 3:   0%|          | 0/101 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:905: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


Running Epoch 2 of 3:   0%|          | 0/101 [00:00<?, ?it/s]

Running Epoch 3 of 3:   0%|          | 0/101 [00:00<?, ?it/s]

(303, 0.24085208094946228)

**5. Evaluation on Validation Set**

In [ ]:
result_bert, model_outputs_bert, wrong_predictions_bert = bert_model.eval_model(val_df)

print("BERT Evaluation Results:")
print(result_bert)

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


0it [00:00, ?it/s]

Running Evaluation:   0%|          | 0/26 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


BERT Evaluation Results:
{'mcc': np.float64(0.9186544584869017), 'eval_loss': 0.2143649046237652}


In [ ]:
result_roberta, model_outputs_roberta, wrong_predictions_roberta = roberta_model.eval_model(val_df)

print("RoBERTa Evaluation Results:")
print(result_roberta)

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


0it [00:00, ?it/s]

Running Evaluation:   0%|          | 0/26 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


RoBERTa Evaluation Results:
{'mcc': np.float64(0.9414751163442803), 'eval_loss': 0.2891411643761855}


**6. Saving the Best Model**

In [ ]:
bert_model.save_model('bert_best_model')

In [ ]:
roberta_model.save_model('roberta_best_model')

**7. Prediction on Real-World Input**

In [ ]:
from simpletransformers.classification import ClassificationModel

bert_model = ClassificationModel('bert', 'yiyanghkust/finbert-tone', use_cuda=False)

real_world_text = ["Global stock market surge as as investors respond positively to the latest economic stimulus pakage,fueling optimism for a strong post-pandemic recovery.", "Oil prices climb to their highest levels in three years,boosting energy sector stocks and signaling strong demand in the global market."]

predictions_bert, _ = bert_model.predict(real_world_text)

print(f"BERT Predictions: {predictions_bert}")

config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

BERT Predictions: [1 1]


In [ ]:
roberta_model = ClassificationModel('roberta', 'roberta-base', use_cuda=False)

real_world_text = ["Global stock market surge as as investors respond positively to the latest economic stimulus pakage,fueling optimism for a strong post-pandemic recovery.", "Oil prices climb to their highest levels in three years,boosting energy sector stocks and signaling strong demand in the global market."]

predictions_roberta, _ = roberta_model.predict(real_world_text)

print(f"RoBERTa Predictions: {predictions_roberta}")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

RoBERTa Predictions: [1 1]


In [ ]:
result_bert, model_outputs_bert, wrong_predictions_bert = bert_model.eval_model(val_df)

print("BERT Evaluation Results:")
print(result_bert)

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


0it [00:00, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

BERT Evaluation Results:
{'mcc': np.float64(0.37012488833845136), 'eval_loss': 5.92933988571167}


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def get_metrics(model, val_df):
    # Extract texts and true labels from the validation dataframe
    texts = val_df['text'].tolist()
    y_true = val_df['label'].tolist()

    # Get model predictions using the model's predict method
    predictions, _ = model.predict(texts)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_true, predictions)
    precision = precision_score(y_true, predictions, average='weighted')
    recall = recall_score(y_true, predictions, average='weighted')
    f1 = f1_score(y_true, predictions, average='weighted')

    return accuracy, precision, recall, f1

# Dictionary to store metrics for both models
metrics_dict = {
    "Model": [],
    "Accuracy": [],
    "Precision": [],
    "Recall": [],
    "F1 Score": []
}


models = [(bert_model, "BERT Model"), (roberta_model, "RoBERTa Model")]

for model, name in models:
    acc, prec, rec, f1 = get_metrics(model, val_df)
    metrics_dict["Model"].append(name)
    metrics_dict["Accuracy"].append(round(acc, 4))
    metrics_dict["Precision"].append(round(prec, 4))
    metrics_dict["Recall"].append(round(rec, 4))
    metrics_dict["F1 Score"].append(round(f1, 4))

# Create and display the results table
results_df = pd.DataFrame(metrics_dict)
print("Evaluation Metrics Comparison:")
display(results_df)

0it [00:00, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]

Evaluation Metrics Comparison:


,Model,Accuracy,Precision,Recall,F1 Score
0,BERT Model,0.9455,0.9464,0.9455,0.9452
1,RoBERTa Model,0.9554,0.9558,0.9554,0.9549
